In [306]:
import pandas as pd
import numpy as np
import regex as re
from io import StringIO
import string
from datetime import datetime


In [307]:
# url = "https://www.sec.gov/Archives/edgar/data/1396440/000155837021001943/tmb-20201231x10k.htm"

# header = {
#   "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
#   "X-Requested-With": "XMLHttpRequest"
# }

# r = requests.get(url, headers=header)

In [308]:
# tables_on_page = pandas.read_html(r.text)

In [309]:
# master_table = None
# i = 0
# for df in tables_on_page:
#     for k in df.columns:
#         try:
#             if(len(k)>1):
#                 if("Company" in k[1]):
#                     print(df)
#                     if master_table is None:
#                         master_table = df
#                     else:
#                         master_table = master_table.append(df.reset_index(drop=True))
#         except:
#             print("Err")

In [310]:
# master_table = master_table.reset_index(drop=True)

In [311]:
# master_table.to_csv("testing_testing.csv")

# NEW

In [312]:
doc = "output-3-pandas.csv"
if('-35-' in doc):
    qtr_date = 'December 31, 2020'
else:
    qtr_date = '22132'
pd_csv = pd.read_csv(doc)
pd_csv = pd_csv.iloc[:, 1:]
# Resetting index
# Find out where portfolio company is
for col in pd_csv.columns:
    pd_csv[col] = pd_csv[col].astype(str).str.replace('  ', ' ')
    portfolio_index_list = pd_csv.index[pd_csv[col].astype(str).str.contains("Company")].to_list()
    if (len(portfolio_index_list) > 0):
        break
portfolio_index = portfolio_index_list[0]
# if(len(portfolio_index_list)>1):
#     pd_csv.drop(portfolio_index_list[1], inplace=True)

# Checking if all the column are in the row with portfolio and adding them if they are not.
for j in range(pd_csv.shape[1]):
    if (pd_csv.iat[portfolio_index - 1, j] == 'Principal'):
        pd_csv.iat[portfolio_index, j] = 'Principal'
    elif (pd_csv.iat[portfolio_index - 1, j] == 'Cost'):
        pd_csv.iat[portfolio_index, j] = 'Cost'
    elif (pd_csv.iat[portfolio_index - 1, j] == 'Fair'):
        pd_csv.iat[portfolio_index, j] = 'Fair Value'
    elif ('Principal' in str(pd_csv.iat[portfolio_index - 1, j])):
        pd_csv.iat[portfolio_index, j] = 'Principal'

print("Portofolio", portfolio_index)
# Removing rows before portfolio index
if (pd_csv.iloc[portfolio_index].str.contains('Company').any() == True):
    pd_csv = pd_csv.iloc[portfolio_index:]


Portofolio 1


In [313]:
pd_csv[-60:]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
270,"North American Breweries Holdings, LLC(10)",NaN,Operator of Specialty Breweries,NaN,"LIBOR Plus 6.25%, Current Coupon 7.50%, Secure...",NaN,NaN,4000.0,NaN,NaN,3921,NaN,NaN,4020.0,NaN
271,"Northland Cable Television, Inc.(10)",NaN,Television Broadcasting,NaN,"LIBOR Plus 6.00%, Current Coupon 7.75%, Secure...",NaN,NaN,4812.0,NaN,NaN,4710,NaN,NaN,4692.0,NaN
272,Oberthur Technologies SA(10)(12),NaN,"Smart Card, Printing, Identity, and Cash Prote...",NaN,"LIBOR Plus 5.00%, Current Coupon 6.25%, Secure...",NaN,NaN,6965.0,NaN,NaN,6648,NaN,NaN,6913.0,NaN
273,Oneida Ltd.(10),NaN,Household Products Manufacturer,NaN,"LIBOR Plus 7.75%, Current Coupon 9.25%, Secure...",NaN,NaN,1933.0,NaN,NaN,1899,NaN,NaN,1904.0,NaN
274,"Panolam Industries International, Inc.(10)",NaN,Decorative Laminate Manufacturer,NaN,"LIBOR Plus 6.00%, Current Coupon 7.25%, Secure...",NaN,NaN,4048.0,NaN,NaN,4010,NaN,NaN,4038.0,NaN
275,"Peppermill Casinos, Inc.(10)",NaN,Operator of Casinos and Gaming Operations,NaN,"LIBOR Plus 6.00%, Current Coupon 7.25%, Secure...",NaN,NaN,2295.0,NaN,NaN,2204,NaN,NaN,2246.0,NaN
276,Phillips Plastic Corporation(10),NaN,Custom Molder of Plastics and Metals,NaN,"LIBOR Plus 5.00%, Current Coupon 6.50%, Secure...",NaN,NaN,1728.0,NaN,NaN,1714,NaN,NaN,1723.0,NaN
277,"Physician Oncology Services, L.P.(10)",NaN,Provider of Radiation Therapy and Oncology Ser...,NaN,"LIBOR Plus 6.25%, Current Coupon 7.75%, Secure...",NaN,NaN,942.0,NaN,NaN,935,NaN,NaN,904.0,NaN
278,PL Propylene LLC(10)(12),NaN,Propylene Producer,NaN,"LIBOR Plus 5.75%, Current Coupon 7.00%, Secure...",NaN,NaN,3970.0,NaN,NaN,3901,NaN,NaN,4035.0,NaN
279,"Preferred Proppants, LLC(10)",NaN,Producer of Sand Based Proppants,NaN,"LIBOR Plus 6.00%, Current Coupon 7.50%, Secure...",NaN,NaN,5942.0,NaN,NaN,5823,NaN,NaN,5526.0,NaN


In [314]:
#Identifying levels in the dataframe
listOfLevels = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if (pd.notnull(pd_csv.iat[i, j])):
            # print(pd_csv.iat[i, j])
            if ((('investments' in str(pd_csv.iat[i, j])) or ('Investments' in str(pd_csv.iat[i, j]))) and
                    ('Subtotal' not in pd_csv.iat[i, j]) and ('Total' not in pd_csv.iat[i, j]) and
                    ('Inc' not in pd_csv.iat[i, j]) and ('Fund' not in pd_csv.iat[i, j]) and 
                    ('LLC' not in pd_csv.iat[i, j]) and ('L.P' not in pd_csv.iat[i, j])  and 
                    ('CMS' not in pd_csv.iat[i, j]) and ('Holdings' not in pd_csv.iat[i, j])  ):
                listOfLevels.append(i)

In [315]:
listOfInvestments = list()

if (pd_csv.iloc[0].str.contains('Type of Investment').any() == False):
    for i in range(pd_csv.shape[0]):
        for j in range(pd_csv.shape[1]):
            if (pd.notnull(pd_csv.iat[i, j])):
                # print(pd_csv.iat[i, j])
                if ((('Senior Secured Notes' in str(pd_csv.iat[i, j])) or
                     ('Senior Secured Second Lien Term Loans' in str(pd_csv.iat[i, j])) or
                     ('Warrants/Equity' in str(pd_csv.iat[i, j])))
                        and ('Total' not in str(pd_csv.iat[i, j]))):
                    listOfInvestments.append(i)
    # Adding the Levels to an array
    InvestmentValues = []
    InvestmentValues.append("Type of Investments")
    InvestmentValues.append("")
    for i in range(len(listOfInvestments)):
        beginIndex = listOfInvestments[i]
        if (i == len(listOfInvestments) - 1):
            endIndex = len(pd_csv)
        else:
            endIndex = listOfInvestments[i + 1]
        value = pd_csv.iat[beginIndex, 0]
        value = ''.join([i for i in value if not i.isdigit()])
        value = re.sub("[^\P{P}-]+", "", value)
        for x in range(beginIndex, endIndex):
            InvestmentValues.append(value)
            # Adding the array to the df
    # pd_csv.insert(1, 'Type of Investments', InvestmentValues)

In [316]:
remove = string.punctuation
remove = remove.replace("-", "") # don't remove hyphens
pattern = r"[{}]".format(remove) # create the pattern

# Adding the Levels to an array
newValues = []
newValues.append("Levels")

for i in range(len(listOfLevels)):
    beginIndex = listOfLevels[i]
    if (i == len(listOfLevels) - 1):
        endIndex = len(pd_csv)
    else:
        endIndex = listOfLevels[i + 1]
    value = pd_csv.iat[beginIndex, 0]
    value = ''.join([i for i in value if not i.isdigit()])
    value = re.sub("[^\P{P}-]+", "", value)
    print("Begin",beginIndex)
    print("End",endIndex)
    for x in range(beginIndex, endIndex):
        newValues.append(value)

#print(newValues)
if(len(newValues)<(len(pd_csv))):
    newValues.insert(0, 'Levels')
# Adding the array to the df
pd_csv.insert(0, 'Levels', newValues)



Begin 1
End 123
Begin 123
End 184
Begin 184
End 329


In [317]:
# Finding Subtotals
pd_csv.reset_index()
subtotal_list = list()
total_list = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if (('Subtotal' in str(pd_csv.iat[i, j]))):
            # print(pd_csv.iloc[i,:])
            subtotal_list.append(i)

print("Subtotal", subtotal_list)
# Dropping the subtotal rows
pd_csv.drop(pd_csv.index[subtotal_list], inplace=True)

# Finding Totals
total_list = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if ('Total' in str(pd_csv.iat[i, j])):
            total_list.append(i)
            
print("Total list",total_list)
pd_csv.drop(pd_csv.index[total_list], inplace=True)
# Dropping everything after the total rows
try:
    pd_csv = pd_csv.iloc[:total_list[-1]]
except:
    print("Total not found")
    
for i in range(5):
    for j in range(pd_csv.shape[1]):
        if ('TypeofInvestment' in str(pd_csv.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            type_of_investment_index = j
            break
        elif ('Industry' in str(pd_csv.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            type_of_investment_index = j
print("type_of_investment_index",type_of_investment_index)

Subtotal [122, 183, 318, 327]
Total list [317, 324]
type_of_investment_index 5


In [318]:
#print(type_of_investment_index)
# Adding company for the middle values where it might be missed out
for i in range(2, pd_csv.shape[0]):
    for j in range(type_of_investment_index):
        row_alphabet_check = False
        if (pd.isnull(pd_csv.iat[i, j]) or (pd_csv.iat[i, j] == 'nan')):
            # print("Index",i,j)
            if (i > 0):
                # print(pd_csv.iat[i-1,j])
                if (pd.isnull(pd_csv.iat[i - 1, j])):
                    1
                else:
                    if (re.search('[a-zA-Z]', str(pd_csv.iat[i - 1, j]))):
                        for k in range(pd_csv.shape[1]):
                            if (re.search('[a-zA-Z]', str(pd_csv.iat[i, k])) and pd.notnull(pd_csv.iat[i, k])):
                                # print("Alpha Check",pd_csv.iat[i,k])
                                row_alphabet_check = True
                                break
                        if (row_alphabet_check):
                            # print(pd_csv.iat[i-1,j])
                            pd_csv.iat[i, j] = pd_csv.iat[i - 1, j]



In [319]:
pd_csv

,Levels,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
1,Levels,Portfolio Company(1),NaN,Business Description,NaN,Type of Investment(2)(3),NaN,Principal(4),NaN,Cost(4),NaN,Fair Value,NaN,NaN,NaN,NaN
2,Control Investments,Control Investments(5),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Control Investments,"Bond-Coat, Inc .",NaN,Casing and Tubing,NaN,"12% Secured Debt (Maturity — December 28, 2017)",NaN,NaN,14750.0,NaN,NaN,14550,NaN,NaN,14550,NaN
4,Control Investments,"Bond-Coat, Inc .",NaN,Coating Services,NaN,Common Stock (Fully diluted 43.4%),NaN,NaN,NaN,NaN,NaN,6350,NaN,NaN,6350,NaN
5,Control Investments,"Bond-Coat, Inc .",NaN,Coating Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,Non-ControlNon-Affiliate Investments,Ceridian Corporation (12),NaN,Investments in Marketable Securities and Diver...,NaN,"LIBOR Plus 5.75%, Current Coupon 5.96%, Secure...",NaN,NaN,10000.0,NaN,NaN,10025,NaN,NaN,10013.0,NaN
324,Non-ControlNon-Affiliate Investments,Compass Investors Inc.(12),NaN,Investments in Marketable Securities and Diver...,NaN,"LIBOR Plus 4.00%, Current Coupon 5.25%, Secure...",NaN,NaN,7000.0,NaN,NaN,7005,NaN,NaN,6994.0,NaN
325,Non-ControlNon-Affiliate Investments,First Data Corporation (12),NaN,Investments in Marketable Securities and Diver...,NaN,"LIBOR Plus 4.00%, Current Coupon 4.21%, Secure...",NaN,NaN,5000.0,NaN,NaN,4763,NaN,NaN,4767.0,NaN
326,Non-ControlNon-Affiliate Investments,"Toll Road Investors Partnership II, LP Bond(12)",NaN,Investments in Marketable Securities and Diver...,NaN,"Zero Coupon Bond (Maturity - February 15, 2033)",NaN,NaN,7500.0,NaN,NaN,1742,NaN,NaN,1834.0,NaN


In [320]:
for i in range(5):
    for j in range(pd_csv.shape[1]):
        if ('Principal' in str(pd_csv.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            principal_index = j
            break
print("Principal Index",principal_index)

Principal Index 7


In [321]:
# Identifying index of maturity
i = 0
# Extracting the column data after shares/warrants/principal
try:
    pd_csv_principal = pd_csv.iloc[:, principal_index:]
    pd_csv_principal = pd_csv_principal.replace("%",np.nan)
    pd_csv_principal = pd_csv_principal.replace("\u200b",np.nan)
    pd_csv_principal = pd_csv_principal.replace("\u200b.1",np.nan)
    
    pd_csv_principal = pd.read_csv(StringIO(re.sub(',+', ',', pd_csv_principal.to_csv())))

except:
    print("no Maturity")

In [322]:
pd_csv_principal

,Unnamed: 0,6,7,8,9,10,11,12,13,14
0,1,Principal(4),Cost(4),Fair Value,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,14750.0,14550,14550,NaN,NaN,NaN,NaN,NaN,NaN
3,4,6350,6350,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
318,323,10000.0,10025,10013.0,NaN,NaN,NaN,NaN,NaN,NaN
319,324,7000.0,7005,6994.0,NaN,NaN,NaN,NaN,NaN,NaN
320,325,5000.0,4763,4767.0,NaN,NaN,NaN,NaN,NaN,NaN
321,326,7500.0,1742,1834.0,NaN,NaN,NaN,NaN,NaN,NaN


In [323]:
pd_csv_principal = pd_csv_principal.iloc[:,1:]
# Dropping columns with Nulls
perc = 95  # Like N %
min_count = int(((100 - perc) / 100) * pd_csv_principal.shape[0] + 1)
pd_csv_principal = pd_csv_principal.dropna(axis=1,
                                           thresh=min_count)

In [324]:
for i in range(5):
    for j in range(pd_csv_principal.shape[1]):
        if ('FairValue' in str(pd_csv_principal.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            fair_index = j
            break
print("Fair Value Index",fair_index)

Fair Value Index 2


In [325]:
pd_csv_principal = pd_csv_principal.iloc[:,:fair_index+1]


In [326]:
pd_csv_principal

,6,7,8
0,Principal(4),Cost(4),Fair Value
1,NaN,NaN,NaN
2,14750.0,14550,14550
3,6350,6350,NaN
4,NaN,NaN,NaN
...,...,...,...
318,10000.0,10025,10013.0
319,7000.0,7005,6994.0
320,5000.0,4763,4767.0
321,7500.0,1742,1834.0


In [327]:
if (qtr_date == 'December 31, 2020'):
    fair_cols = [col for col in pd_csv.columns if 'Fair' in col]
    pd_csv = pd_csv.reset_index(drop=True)

    for i in range(int(pd_csv_principal.shape[0])):
        for j in range(int(pd_csv_principal.shape[1])):
            #print(i)
            #print(pd_csv.iat[i,j])
            if(pd.isnull(pd_csv_principal.iat[i,j])):
                1
                #print("")
            elif('-' in pd_csv_principal.iat[i,j]):
                if(len(pd_csv_principal.iat[i,j])>1):
                    pd_csv_principal.iat[i,j] = np.nan

In [328]:
for i in range(int(pd_csv_principal.shape[0])):
    for j in range(pd_csv_principal.shape[1]):
        try:
            pd_csv_principal.iat[i,j] = re.sub(r"\(\d+\)", "", pd_csv_principal.iat[i,j])
            if(pd_csv_principal.iat[i,j] == ''):
                pd_csv_principal.iat[i,j] = np.nan
        except:
            print("Bracket digits not removed")

Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
B

In [329]:
pd_csv_principal

,6,7,8
0,Principal,Cost,Fair Value
1,NaN,NaN,NaN
2,14750.0,14550,14550
3,6350,6350,NaN
4,NaN,NaN,NaN
...,...,...,...
318,10000.0,10025,10013.0
319,7000.0,7005,6994.0
320,5000.0,4763,4767.0
321,7500.0,1742,1834.0


In [330]:
try:
    last_1_nan_list = pd_csv_principal.index[(pd_csv_principal.iloc[:, -1].isnull()) & (pd_csv_principal.iloc[:, -2].notnull()) & (
        pd_csv_principal.iloc[:, -3].notnull())].to_list()
    pd_csv_principal.loc[last_1_nan_list] = pd_csv_principal.loc[last_1_nan_list].shift(1, axis=1)
except:
    print("No Rate")

In [331]:
pd_csv_principal[:60]

,6,7,8
0,Principal,Cost,Fair Value
1,NaN,NaN,NaN
2,14750.0,14550,14550
3,NaN,6350,6350
4,NaN,NaN,NaN
5,NaN,20900,20900
6,500.0,500,500
7,NaN,42,3690
8,NaN,NaN,NaN
9,NaN,542,4190


In [332]:
pd_csv = pd_csv.iloc[:, :principal_index].reset_index(drop=True)
# Concatting everything together
pd_csv = pd.concat([pd_csv, pd_csv_principal], axis=1)


In [333]:
perc = 100  # Like N %
min_count = int(((100 - perc) / 100) * pd_csv.shape[0] + 1)
# Dropping columns which have 100% NaN
pd_csv = pd_csv.dropna(axis=1,
                       thresh=min_count)

new_header = pd_csv.iloc[0]  # grab the first row for the header
#new_header = new_header.str.replace("Shares/Warrants/Principal","Shares Warrants Principal")
pd_csv = pd_csv[1:]  # take the data less the header row
pd_csv.columns = new_header.str.replace(r"\(\d+\)", "")  # set the header row as the df header

C:\Users\jaiso\AppData\Local\Temp\ipykernel_11508\4052734264.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  pd_csv.columns = new_header.str.replace(r"\(\d+\)", "")  # set the header row as the df header


In [334]:
pd_csv = pd_csv.replace("~", ",", regex=True)
#pd_csv = pd_csv.iloc[:, 1:]

# Dropping columns which NaN as column names
pd_csv = pd_csv.loc[:, pd_csv.columns.notna()]
maturity_cols = [col for col in pd_csv.columns if 'Maturity' in col]
if(len(maturity_cols)<1):
    pd_csv = pd_csv.dropna(thresh=len(pd_csv.columns) - 3)
else:
    pd_csv = pd_csv.dropna(thresh=len(pd_csv.columns) - 5)

In [335]:
perc = 95  # Like N %
min_count = int(((100 - perc) / 100) * pd_csv.shape[0] + 1)
pd_csv = pd_csv.dropna(axis=1,thresh=min_count)

In [336]:
pd_csv.isnull().sum()/len(pd_csv)

0
Levels                  0.000000
Portfolio Company       0.000000
Business Description    0.000000
Type of Investment      0.185430
Principal               0.463576
Cost                    0.009934
Fair Value              0.009934
dtype: float64

In [337]:
try:
    pd_csv.drop('​.1', inplace=True, axis=1)
except:
    print(".1 column not present")

.1 column not present


In [338]:
pd_csv

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
2,Control Investments,"Bond-Coat, Inc .",Casing and Tubing,"12% Secured Debt (Maturity — December 28, 2017)",14750.0,14550,14550
3,Control Investments,"Bond-Coat, Inc .",Coating Services,Common Stock (Fully diluted 43.4%),NaN,6350,6350
5,Control Investments,"Bond-Coat, Inc .",Coating Services,NaN,NaN,20900,20900
6,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,"12% Secured Debt (Maturity — April 20, 2013)",500.0,500,500
7,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,Member Units (Fully diluted 41.0%)(8),NaN,42,3690
...,...,...,...,...,...,...,...
318,Non-ControlNon-Affiliate Investments,Ceridian Corporation (12),Investments in Marketable Securities and Diver...,"LIBOR Plus 5.75%, Current Coupon 5.96%, Secure...",10000.0,10025,10013.0
319,Non-ControlNon-Affiliate Investments,Compass Investors Inc.(12),Investments in Marketable Securities and Diver...,"LIBOR Plus 4.00%, Current Coupon 5.25%, Secure...",7000.0,7005,6994.0
320,Non-ControlNon-Affiliate Investments,First Data Corporation (12),Investments in Marketable Securities and Diver...,"LIBOR Plus 4.00%, Current Coupon 4.21%, Secure...",5000.0,4763,4767.0
321,Non-ControlNon-Affiliate Investments,"Toll Road Investors Partnership II, LP Bond(12)",Investments in Marketable Securities and Diver...,"Zero Coupon Bond (Maturity - February 15, 2033)",7500.0,1742,1834.0


In [339]:
## Switch unless there is Maturity Date in the picture
if(len(maturity_cols)<1):
    last_2_as_nan_list = pd_csv.index[(pd_csv.iloc[:,-3].isnull()) & (pd_csv.iloc[:,-4].isnull())  & (pd_csv.iloc[:,-5].notnull())].to_list()
    #Need to drop last 2 NaN List
    pd_csv.drop(last_2_as_nan_list,inplace=True)
else:
    print("Maturity column present")

In [340]:
#Code for the overflowing business description
pd_csv = pd_csv.reset_index(drop=True)

comp_cols = [col for col in pd_csv.columns if 'Company' in col]
desc_cols = [col for col in pd_csv.columns if 'Description' in col]

business_desc_index = list()
business_desc_temp = ''
i = 0
while(i < len(pd_csv)-1):
    if( pd_csv.loc[i,comp_cols[0]] == pd_csv.loc[i+1,comp_cols[0]] ):
        if((pd_csv.loc[i,desc_cols[0]] != pd_csv.loc[i+1,desc_cols[0]])):
            company_name = pd_csv.loc[i,comp_cols[0]]
            k = 0
            business_desc_temp = ''
            #print(company_name)
            while ( k != -1 and k+i+1<len(pd_csv)):
                #print("K",k)
                if(company_name == pd_csv.loc[k+i,comp_cols[0]]):
                    #print("Company Name",pd_csv.loc[k+i,comp_cols[0]])
                    if(str(pd_csv.loc[k+i,desc_cols[0]]) not in business_desc_temp):
                        business_desc_temp = business_desc_temp+' '+str(pd_csv.loc[k+i,desc_cols[0]])
                    #print(business_desc_temp)
                else:
                    break
                k = k + 1
            index_run = 0
            while(index_run < k):
                pd_csv.loc[index_run+i,desc_cols[0]] = business_desc_temp
                index_run = index_run + 1
            i = i + k
    i = i + 1

In [341]:
#last 2 nan
pd_csv = pd_csv.reset_index(drop=True)
try:
    last_2_nan_list = pd_csv.index[(pd_csv.iloc[:, -1].isnull()) & (pd_csv.iloc[:, -2].isnull())].to_list()
    #pd_csv_principal.loc[last_2_nan_list] = pd_csv_principal.loc[last_1_nan_list].shift(1, axis=1)
except:
    print("No Nulls")

In [342]:
pd_csv.columns

Index(['Levels', 'Portfolio Company', 'Business Description',
       'Type of Investment', 'Principal', 'Cost', 'Fair Value'],
      dtype='object', name=0)

In [343]:
for i in last_2_nan_list:
    print(pd_csv.loc[i-1])

0
Levels                                                Control Investments
Portfolio Company            Hawthorne Customs and Dispatch Services, LLC
Business Description     Facilitator of Import Logistics, Brokerage, a...
Type of Investment                  Member Units (Fully diluted 47.6%)(8)
Principal                                                             NaN
Cost                                                                  589
Fair Value                                                           1140
Name: 21, dtype: object
0
Levels                                              Affiliate Investments
Portfolio Company                                              SYNEO, LLC
Business Description     Manufacturer of Specialty Cutting Tools and P...
Type of Investment            10% Secured Debt (Leadrock Properties, LLC)
Principal                                                          1440.0
Cost                                                                 1413
Fair Value

In [344]:
last_nan = list()
pd_csv = pd_csv.reset_index(drop=True)

try:
    for i in range(int(pd_csv.shape[0])):
        if(pd_csv.iat[i,pd_csv.shape[1]-1] == ' ' or pd_csv.iat[i,pd_csv.shape[1]-1] == '  '):
            last_nan.append(i)
            pd_csv.iat[i,pd_csv.shape[1]-1] = np.nan
    #pd_csv.loc[last_nan, maturity_cols[0]:] = pd_csv.loc[last_nan, maturity_cols[0]:].shift(1, axis=1)
except:
    print("Error while switching for last column")

In [345]:
pd_csv.loc[last_nan]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value


In [346]:
pd_csv = pd_csv.fillna("")

In [347]:
pd_csv[:60]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
0,Control Investments,"Bond-Coat, Inc .",Casing and Tubing Coating Services,"12% Secured Debt (Maturity — December 28, 2017)",14750.0,14550,14550
1,Control Investments,"Bond-Coat, Inc .",Casing and Tubing Coating Services,Common Stock (Fully diluted 43.4%),,6350,6350
2,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,"12% Secured Debt (Maturity — April 20, 2013)",500.0,500,500
3,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,Member Units (Fully diluted 41.0%)(8),,42,3690
4,Control Investments,"California Healthcare Medical Billing, Inc .",Outsourced Billing and Revenue Cycle Management,"12% Secured Debt (Maturity — October 17, 2015)",8103.0,7913,8016
5,Control Investments,"California Healthcare Medical Billing, Inc .",Outsourced Billing and Revenue Cycle Management,Warrants (Fully diluted 21.3%),,1193,3380
6,Control Investments,"California Healthcare Medical Billing, Inc .",Outsourced Billing and Revenue Cycle Management,Common Stock (Fully diluted 9.8%),,1177,1560
7,Control Investments,"CBT Nuggets, LLC",Produces and Sells IT,"14% Secured Debt (Maturity — December 31, 2013)",450.0,450,450
8,Control Investments,"CBT Nuggets, LLC",Training Certification,Member Units (Fully diluted 41.6%)(8),,1300,7800
9,Control Investments,"Ceres Management, LLC (Lambs)",Aftermarket Automotive Services Chain,"14% Secured Debt (Maturity — May 31, 2013)",4000.0,3993,3993


In [348]:
#df_xlsx = pd.read_excel('MAIN_Final_Test_2.xlsx', sheet_name=None)

In [349]:
#df_xlsx_1 = pd.read_excel('MAIN_Final_Test.xlsx', sheet_name=None)

In [350]:
#Loop through the sheets and print the number of rows
# for sheet_name in df_xlsx.keys():
#     df = df_xlsx[sheet_name]
#     df_2 = df_xlsx_1[sheet_name]
#     print(f"{sheet_name} has {len(df.index)} rows in df1 and {len(df_2.index)} in df 2")